# Responses to MusicLab via Nettskjema API
The current version of the MusicLab app (0.0.17) submits mobile sensor data to Nettskjema in one minute packages. This format is great for ensuring we get as much data as possible before something goes wronge with the app or device, but it is not convenient for analysis or storage. This notebook presents the code for extracting and rearranging a large number of MusicLab recordings in an automated fashion. This is written specifically to handle the Copenhangen MusicLab concert of October 26th, 2021, however the shape should be useful for future projects as well. 

In [4]:
import requests
import json
import zipfile
import socket
import os
import base64
import time
import shutil

import pandas as pd

First loading up the API token

In [5]:
os.listdir()

['PullingMusicLab.ipynb',
 '.DS_Store',
 'PullingNettskjema.ipynb',
 'data.zip',
 'CompressedData',
 'README.md',
 '.gitignore',
 'copen_netts.tsv',
 'Test_API',
 '.ipynb_checkpoints',
 'Copendata',
 '.git',
 'InstOrdData',
 'nettskjema_token.txt']

In [6]:
f=open('nettskjema_token.txt','r')
TOKEN = f.read()
f.close()

In [7]:
session = requests.Session()
session.headers.update({'Authorization': 'Bearer ' + TOKEN})

Test the token by sending a request to see its expiration date.

In [8]:
# confirm the token is working on this IP with the check on expiry date
request_url = "https://nettskjema.no/api/v2/users/admin/tokens/expire-date"
response = session.get(request_url)
response.content

b'{"expireDate":"2022-10-18T17:38:29.000+0200"}'

# Gathering Musiclab phone sensor data


In [9]:
formID = 141510
submissionID = 16561653
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID) + '/submissions?fromSubmissionId=' + str(submissionID)
response = session.get(request_url)
subIDs = json.loads(response.content.decode())
print('Submissions since subID ' + str(submissionID) + ': ' + str(len(subIDs)))
if len(subIDs)<5: # print error message or top responses 
    print(subIDs)
else:
    print(subIDs[0])


Submissions since subID 16561653: 23523
{'submissionId': 16794081, 'createdDate': '2021-10-31T20:11:18.000+0100', 'modifiedDate': '2021-10-31T20:11:18.000+0100', 'delivered': True, 'answerTime': 0, 'answers': [{'answerId': 103297368, 'questionId': 1996787, 'textAnswer': 'fb5a1b4d-e5c4-638c-1a08-932a34fd07b4'}, {'answerId': 103297367, 'questionId': 1996788, 'textAnswer': 'data.zip', 'attachments': [{'answerAttachmentId': 302899, 'fileName': 'data.zip', 'mediaType': 'application/zip'}]}]}


In [62]:
# test getting metadata on submissions for the music lab app. Here used to find a submission ID corresponding to 
# specific recording times
subIDs[-2005]

{'submissionId': 16631898,
 'createdDate': '2021-10-26T02:35:51.000+0200',
 'modifiedDate': '2021-10-26T02:35:52.000+0200',
 'delivered': True,
 'answerTime': 0,
 'answers': [{'answerId': 102496820,
   'questionId': 1996788,
   'textAnswer': 'data.zip',
   'attachments': [{'answerAttachmentId': 279513,
     'fileName': 'data.zip',
     'mediaType': 'application/zip'}]},
  {'answerId': 102496819,
   'questionId': 1996787,
   'textAnswer': 'de8b04e5-9db8-056d-0105-8adf1b676dfd'}]}

in current subID list:
subIDs[2565] = 
 'submissionId': 16672589,
 'createdDate': '2021-10-26T22:29:56.000+0200'
 
subIDs[-3171] = 
'submissionId': 16651235,
 'createdDate': '2021-10-26T19:30:00.000+0200',
 
 subIDs[-2380] = 
 'submissionId': 16648859,
 'createdDate': '2021-10-26T18:26:55.000+0200',
 
subIDs[-2005]
'submissionId': 16631898,
 'createdDate': '2021-10-26T02:35:51.000+0200'
 
 

# Gather app recordings from the day of the copenhagen concert
Download the files from nettskjema and then reorganise

In [15]:
# prep folder
os.mkdir('./Copendata')
os.chdir('Copendata')

In [69]:
# one practice call using subID collected above
subn = 0 # just calling one as an example

subID = str(subIDs[subn]['submissionId'])
attID = str(subIDs[subn]['answers'][0]['attachments'][0]['answerAttachmentId'])
request_url = 'https://nettskjema.no/api/v2/submissions/' + subID + '/attachments/' + attID
response = session.get(request_url)

att_dets = json.loads(response.content.decode())

# write the decoded attachment into a zip file
f=open('data.zip', 'wb')
f.write(base64.b64decode(att_dets['content']))
f.close()

# and then unzip that file, leaving a uniquely titled csv, I hope
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    if not os.path.exists(str(subID)):
        os.mkdir(str(subID))
        zip_ref.extractall('./'+str(subID)) # Not unique filenames so use the unique submission IDs 

print(os.listdir())
os.chdir('./'+str(subID))
print(os.listdir())
os.chdir('..')

['data.zip', '16767882']
['cfcd73d7-4af9-08e2-aa6a-e73e87bbfbf3.deviceMotion.csv']


Here we have a minute recording from a device with the unique installation ID 'cfcd73d7-4a...' in a format that is easy to read. 

The files within the zip are named for the device and information type, but do not include the submission number. This means if they are opened into a folder that already contains a previous recording from the same device, the previously saved recording will be overwritten and lost. To avoid this, the files are unzipped within a folder names for that unique submission. 

Now to collect many at once: 

In [9]:
# os.chdir('CopenData')

In [28]:
subs_already  = os.listdir()#.sort()
subs_already.sort()
print(len(subs_already))
lastsub = '16609378' #subs_already[-2]
lastsub 
# 16550625

2274


'16609378'

In [10]:
# collect submission IDs to form from a specific date
#form_ID = 141510
#submissions_ID_range = 'https://nettskjema.no/api/v2/forms/' + str(form_ID) + '/submissions?fields=submissionId&fromSubmissionId=' + lastsub
submissions_ID_range = 'https://nettskjema.no/api/v2/forms/141510/submissions?fields=submissionId&fromDate=2021-10-25T13%3A43%3A17.486%2B0100'

response = session.get(submissions_ID_range)
subIDs = json.loads(response.content.decode())#eval(response.content.decode())
print(len(subIDs))
if len(subIDs)<5:
    print(subIDs)
else:
    print(subIDs[0])
    
#1976
# {'submissionId': 16609378}

354
{'submissionId': 16638490}


In [17]:
# pull in all the data as fast as possible.
submission_dets = 'https://nettskjema.no/api/v2/submissions/'
newSubs = 0
tic = time.time()
for subm in subIDs: #[1000:]: # restrict range as necessary for testing
    # first find out the attachment file ID for this submission
    subid = subm["submissionId"]
    response = session.get(submission_dets + str(subid) + '/attachments')
    sub_atts = json.loads(response.content.decode())
    # if there is an IDed attachment for this submission, get the file
    if len(sub_atts)>0:
        attid = sub_atts[0]
        newSubs += 1
        response = session.get(submission_dets + str(subid) + '/attachments/' + str(attid))
        att_dets = json.loads(response.content.decode())
        # write the decoded attachment into a zip file
        f=open('data.zip', 'wb')
        f.write(base64.b64decode(att_dets['content']))
        f.close()
        # and then unzip that file, leaving a uniquely titled csv, I hope
        with zipfile.ZipFile('data.zip', 'r') as zip_ref:
            if not os.path.exists(str(subid)):
                os.mkdir(str(subid))
                zip_ref.extractall('./'+str(subid)) # if not unique can use the unique submission IDs 
print(time.time() - tic)
print(newSubs)

4047.5774102211
22503


# Rearrange files for more conveninent processing
Instead of preserving thousands of folders with only one or two samples in each, the original data will be rearranged and files renamed to group recordings by installationID.

Responses will also be grouped into two recording periods: pre-concert measurements and concert measurements  (according to when the responses were recieved by nettskjema).

### Preconcert interval
measurements begining from '2021-10-26T18:26:55.000+0200', the first recording of activity that could occured at the concert hall, when volunteers started helping participants prepare their mobiles for recordings. This interval ends at the intended start time of the concert, '2021-10-26T19:30:00.000+0200'. This corresponds to the submission ID range of [16648859,16651235).

### Concert Interval
From the start of intended concert begining, '2021-10-26T19:30:00.000+0200', to a little bit after the performance had ended, '2021-10-26T22:29:56.000+0200'. (This may be half an hour or 20 minutes after the concert. Need to confirm with time stamps from audio recording.) Corresponding submission ID numbers: [16651235,16672589].

### Earlier and Later Sensor readings
The MusicLab app also recieved data earlier on the 26th, during a pilot test while Ritmo researchers were in the audience for another experiment. There is also data from whenever users turned on the app out of curiousity, or while preparing for the concert. These may be interesting methodologically but will not be retained here. 
 

In [63]:
pwd

'/Users/finn/Desktop/Current_Projects/Copen/Pulling_nettskjema'

In [64]:
os.chdir('Copendata/')


In [67]:
subs = os.listdir()
subs.sort()
subs=subs[1:-1]
print(len(subs))
print([subs[0],subs[-1]])
subs_s = pd.Series(subs)

23505
['16562398', '16767882']


In [137]:
os.chdir('..')

In [69]:
interval_folder = 'PreConcert_MusicLab'
sub_range = ['16648859','16651235']
os.mkdir(interval_folder)

In [87]:
subs_c = subs_s.loc[subs_s >= sub_range[0]]
subs_pc = subs_c.loc[subs_c < sub_range[1]]
print(len(subs_pc))
print([subs_pc.iloc[0],subs_pc.iloc[-1]])
interval_submissions = subs_pc.values

791
['16648859', '16651234']


In [88]:
# copy submissions from subfolders of the Copendata folder to device folders within the interval_folder
foldlist = os.listdir('./' + interval_folder + '/')
tic = time.time()
for subid in interval_submissions:
    filenames = os.listdir('./Copendata/'+str(subid))
    for fn in filenames:
        if fn.endswith('.csv'):
            # extract the installation ID 
            subdets = fn.split('.')
            instid = subdets[0]
            # if the device doesn't have a folder, generate one
            if instid not in foldlist:
                os.mkdir('./' + interval_folder + '/' + instid)
                foldlist = os.listdir('./' + interval_folder + '/')

            sourcefile = './Copendata/' + str(subid) + '/' + fn
            targetfile = './' + interval_folder + '/' + instid + '/'+str(subid) + '.' + fn # always add the subID to filename to ensure uniqueness
            #print(targetfile)
            shutil.copy2(sourcefile,targetfile)
            
print(time.time() - tic)
foldlist

1.2224538326263428


['49d38fe8-2daf-4dc0-8297-5a872da374fb',
 'f20847af-4c00-639e-5484-d705bd7a3437',
 '77451c9f-ea66-2d70-21ac-c2c97a049b79',
 'abf053b1-19ab-912a-60a2-52d43c1fb79f',
 'f9945bf5-e47a-a404-3f45-d6954e78ce2a',
 '31f4e1a5-be17-3a37-ee86-1e215ffcb996',
 '05d05a78-b18c-0928-cbf7-b5b1b9600e55',
 'bfc2274e-acb2-21af-16d5-6ae9fac1593e',
 'c7abdacc-8973-b681-5fd0-c55857b08950',
 '881e7bc9-1dd8-7c3b-9e58-0050948457d8',
 '1a9a903b-9524-f8c2-a943-81a108880e46',
 '8d2b149e-a9a8-12c1-d9f1-816d785730c4',
 'cd1ae6f8-b8cf-4a62-48a2-ff2590e4e4df',
 'd76d0617-df22-e5fa-ddf1-1590723bf307',
 '6a917350-972c-f470-86e4-f4d548a6eddb',
 'd7b52b80-4215-a4fa-99b8-7293c650ea7d',
 '3ee0563f-807c-89a7-0a65-71557f6a9775',
 '9197305e-476d-1858-10a4-9658a5333b19',
 '7406434c-c4d4-8f34-9895-5fb845cb9cb8',
 '93314249-e4d7-9325-5aca-c1a6a0883bcd',
 '3bcb5d6e-b345-47a8-975d-ffcc14ba0a00',
 'acd48012-ac19-a6d7-aaa2-3df76850ded0',
 'b75d0d85-7d79-3483-f0c5-890a1bf93e43',
 'b140038b-ab82-7d7f-4710-f235a96dc4a5',
 'fe0534f2-aaad-

In [89]:
len(foldlist)

111

In [90]:
interval_folder = 'Concert_MusicLab'
sub_range = ['16651235','16672589']
os.mkdir(interval_folder)

In [91]:
subs_c = subs_s.loc[subs_s >= sub_range[0]]
subs_pc = subs_c.loc[subs_c < sub_range[1]]
print(len(subs_pc))
print([subs_pc.iloc[0],subs_pc.iloc[-1]])
interval_submissions = subs_pc.values

17787
['16651235', '16672585']


In [92]:
# copy submissions from subfolders of the Copendata folder to device folders within the interval_folder
foldlist = os.listdir('./' + interval_folder + '/')
tic = time.time()
for subid in interval_submissions:
    filenames = os.listdir('./Copendata/'+str(subid))
    for fn in filenames:
        if fn.endswith('.csv'):
            # extract the installation ID 
            subdets = fn.split('.')
            instid = subdets[0]
            # if the device doesn't have a folder, generate one
            if instid not in foldlist:
                os.mkdir('./' + interval_folder + '/' + instid)
                foldlist = os.listdir('./' + interval_folder + '/')

            sourcefile = './Copendata/' + str(subid) + '/' + fn
            targetfile = './' + interval_folder + '/' + instid + '/'+str(subid) + '.' + fn # always add the subID to filename to ensure uniqueness
            #print(targetfile)
            shutil.copy2(sourcefile,targetfile)
            
print(time.time() - tic)
foldlist

27.735976934432983


['e6a57ce9-75de-1817-c8bb-87d42bea2a55',
 '49d38fe8-2daf-4dc0-8297-5a872da374fb',
 'f20847af-4c00-639e-5484-d705bd7a3437',
 '2c213490-5bad-6fe9-3c7e-db7fbcf99877',
 '77451c9f-ea66-2d70-21ac-c2c97a049b79',
 'abf053b1-19ab-912a-60a2-52d43c1fb79f',
 'f9945bf5-e47a-a404-3f45-d6954e78ce2a',
 '31f4e1a5-be17-3a37-ee86-1e215ffcb996',
 '05d05a78-b18c-0928-cbf7-b5b1b9600e55',
 'bfc2274e-acb2-21af-16d5-6ae9fac1593e',
 'c7abdacc-8973-b681-5fd0-c55857b08950',
 '881e7bc9-1dd8-7c3b-9e58-0050948457d8',
 '1a9a903b-9524-f8c2-a943-81a108880e46',
 'd7e89756-ada2-aed6-b3f7-716160c42fb9',
 'cd1ae6f8-b8cf-4a62-48a2-ff2590e4e4df',
 '090eb70c-5cc5-89d6-8f6c-14b2fdf7ab24',
 '6a917350-972c-f470-86e4-f4d548a6eddb',
 'cbda4462-d6ad-c79b-e925-f686517e4fe3',
 'c5a895bf-3002-6ccf-e7af-dd3a370b4f2f',
 '3ee0563f-807c-89a7-0a65-71557f6a9775',
 '7183296a-8e1e-3ab2-3d85-1bf8badc45d3',
 '9197305e-476d-1858-10a4-9658a5333b19',
 'eda593d9-8906-1f23-f35a-2c2698c19d84',
 '25decfa4-d6e0-d757-722e-0f9db423fc2d',
 '7406434c-c4d4-

In [93]:
print(len(foldlist))

158


# Separate live stream and concert audience
now it would help to know which measurements were from the audience in the concert hall and which were from the live stream. These can be distinguished by responses to the participant ID form: 225781,	0 Physically Present Participant Number. (This can also be confirmed with the device geolocation data, if that is available.)


In [94]:
formID = 225781
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID)
response = session.get(request_url) # using the request session call which includes the saved API token
form_metadata = json.loads(response.content.decode()) # inteprete recieved string into a python native datatype
form_metadata # show the information output

{'formId': 225781,
 'languageCode': 'en',
 'title': '0 Physically Present Participant Number',
 'deliveryDestination': 'DATABASE',
 'formType': 'DEFAULT',
 'theme': 'DEFAULT',
 'createdBy': {'personId': 616293,
  'username': 'danasw@uio.no',
  'fullName': 'Dana Swarbrick',
  'name': 'Dana Swarbrick',
  'type': 'LOCAL'},
 'modifiedBy': {'personId': 1676908,
  'username': 'finnu@uio.no',
  'fullName': 'Finn Upham',
  'name': 'Finn Upham',
  'type': 'LOCAL'},
 'createdDate': '2021-10-21T09:39:49.000+0200',
 'modifiedDate': '2021-10-28T16:29:27.000+0200',
 'openFrom': '2021-10-27T09:44:45.000+0200',
 'respondentGroup': 'ALL',
 'editorsContactEmail': 'danasw@uio.no',
 'editorsSubmissionEmailType': 'NONE',
 'editors': [{'personId': 1914058,
   'username': 'finnu1@api',
   'fullName': 'finn ritmo access',
   'name': 'finn ritmo access',
   'type': 'API'},
  {'personId': 1676908,
   'username': 'finnu@uio.no',
   'fullName': 'Finn Upham',
   'name': 'Finn Upham',
   'type': 'LOCAL'},
  {'perso

In [95]:
formID = 225781
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID) + '/submissions' 
response = session.get(request_url) # using the request session call which includes the saved API token
sub_metadata = json.loads(response.content.decode()) # inteprete recieved string into a python native datatype
sub_metadata[:2] # examples of the last two responses recieved

[{'submissionId': 16674015,
  'createdDate': '2021-10-27T00:00:00.000+0200',
  'modifiedDate': '2021-10-27T00:00:00.000+0200',
  'delivered': True,
  'answerTime': 0,
  'answers': [{'answerId': 102748641,
    'questionId': 3741787,
    'textAnswer': 'Finn'},
   {'answerId': 102748640,
    'questionId': 3741786,
    'textAnswer': '40bf6287-e5a4-8a30-8863-cc71c9ef7225'}]},
 {'submissionId': 16653694,
  'createdDate': '2021-10-26T00:00:00.000+0200',
  'modifiedDate': '2021-10-26T00:00:00.000+0200',
  'delivered': True,
  'answerTime': 0,
  'answers': [{'answerId': 102664391,
    'questionId': 3741786,
    'textAnswer': '08ff9d8a-6bc2-897f-24d3-91eb10717741'}]}]

In [99]:
# gather details

instIDs = []
partIDs = []
subIDs = []
subTime = []

subid = ''
subt = ''
partid = ''
instid = ''

for sub_md in sub_metadata:
    subid = sub_md['submissionId']
    subt = sub_md['createdDate']
    for ans in sub_md['answers']:
        if ans['questionId'] == 3741787:
            partid = ans['textAnswer']
        if ans['questionId'] == 3741786:
            instid = ans['textAnswer']
    instIDs.append(instid)
    partIDs.append(partid)
    subIDs.append(subid)
    subTime.append(subt)
    
    subid = ''
    subt = ''
    partid = ''
    instid = ''
    
participantIDs = pd.DataFrame()
participantIDs['Participants'] = partIDs
participantIDs['Installations'] = instIDs
participantIDs['Submissions'] = subIDs
participantIDs['SubmissionTimes'] = subTime


,Participants,Installations,Submissions,SubmissionTimes
0,Finn,40bf6287-e5a4-8a30-8863-cc71c9ef7225,16674015,2021-10-27T00:00:00.000+0200
1,,08ff9d8a-6bc2-897f-24d3-91eb10717741,16653694,2021-10-26T00:00:00.000+0200
2,,689d4b0e-6e58-049a-5329-e448a3011643,16653300,2021-10-26T00:00:00.000+0200
3,DSQe200,ddfec0ee-dcc2-db2c-0de2-a22dfe78c044,16652352,2021-10-26T00:00:00.000+0200
4,DSQe201,cbda4462-d6ad-c79b-e925-f686517e4fe3,16652284,2021-10-26T00:00:00.000+0200
5,DSQe031,25decfa4-d6e0-d757-722e-0f9db423fc2d,16651361,2021-10-26T00:00:00.000+0200
6,,25f41e25-f069-eca9-607a-3d2a65481167,16651074,2021-10-26T00:00:00.000+0200
7,DSqd029,005093c6-7880-8728-1256-26822f05cfb0,16651006,2021-10-26T00:00:00.000+0200
8,DSQd028,c32d3e7d-8ee6-0a11-b056-cb501edbfc7e,16650970,2021-10-26T00:00:00.000+0200
9,DSQd028,c32d3e7d-8ee6-0a11-b056-cb501edbfc7e,16650960,2021-10-26T00:00:00.000+0200


In [105]:
parts = participantIDs.sort_values('Participants')
parts.iloc[:60]

,Participants,Installations,Submissions,SubmissionTimes
1,,08ff9d8a-6bc2-897f-24d3-91eb10717741,16653694,2021-10-26T00:00:00.000+0200
2,,689d4b0e-6e58-049a-5329-e448a3011643,16653300,2021-10-26T00:00:00.000+0200
116,,40bf6287-e5a4-8a30-8863-cc71c9ef7225,16647948,2021-10-26T00:00:00.000+0200
121,,4e74e5b7-aa5b-3fba-2284-89bd559d8b02,16563374,2021-10-21T00:00:00.000+0200
6,,25f41e25-f069-eca9-607a-3d2a65481167,16651074,2021-10-26T00:00:00.000+0200
40,,f9945bf5-e47a-a404-3f45-d6954e78ce2a,16649891,2021-10-26T00:00:00.000+0200
39,,f9945bf5-e47a-a404-3f45-d6954e78ce2a,16649903,2021-10-26T00:00:00.000+0200
31,,f6437dd9-96b3-7bce-42ef-4f01a6bfde18,16649959,2021-10-26T00:00:00.000+0200
119,1,28790b97-7329-f733-29a7-42a74a35558c,16599602,2021-10-24T00:00:00.000+0200
122,120,7e7716d5-77ef-a28a-c678-3223b5a6b62d,16562808,2021-10-21T00:00:00.000+0200


In [124]:
partNs = [10,110,26,8,113,115,52,50,64,32,108,109,97,98,87,83,71,59,60,56,54,42,65,47,25,49,51,48,44,28,38,22,17,12,11,21,29,100,106,107,24,101,91,81,90,79,95,99,78,75,66,80,69,55,43,35,37,14,16,18,19,5,3,4,7,93,46,92,112,103,86,68,67,70,63,57,30,61,82,72,41,85,77,74]
print(len(partNs))
print(len(parts.loc[partNs,'Installations'].unique()))
print(len(parts.loc[partNs,'Participants'].unique()))
# conflict between DSQE017, 16648940 and DSQd063, 16649474, different participant IDs from same installation. Take the later submission so drop 110
partNs = [10,26,8,113,115,52,50,64,32,108,109,97,98,87,83,71,59,60,56,54,42,65,47,25,49,51,48,44,28,38,22,17,12,11,21,29,100,106,107,24,101,91,81,90,79,95,99,78,75,66,80,69,55,43,35,37,14,16,18,19,5,3,4,7,93,46,92,112,103,86,68,67,70,63,57,30,61,82,72,41,85,77,74]
print(len(partNs))
print(len(parts.loc[partNs,'Installations'].unique()))
print(len(parts.loc[partNs,'Participants'].unique()))

84
83
84
83
83
83


In [126]:
# output csv mapping between participant numbers and installations
df = parts.loc[partNs,:].sort_values('Participants')
df.to_csv('Concert_participants_installations.csv', index=False)

In [132]:
# manually cleaned up ID numbers and then reload and resort
IDs = pd.read_csv('Concert_participants_installations.csv').sort_values('Participants')
IDs

,Participants,Installations,Submissions,SubmissionTimes
1,DSQd001,98ae5c8a-1fed-7d15-a922-9cc2bd9da9e0,16649982,2021-10-26T00:00:00.000+0200
2,DSQd028,c32d3e7d-8ee6-0a11-b056-cb501edbfc7e,16650970,2021-10-26T00:00:00.000+0200
63,DSQd029,005093c6-7880-8728-1256-26822f05cfb0,16651006,2021-10-26T00:00:00.000+0200
3,DSQd036,f045e08d-53b9-5e26-ad38-107564d919bc,16648810,2021-10-26T00:00:00.000+0200
4,DSQd037,77d1745b-a57f-d560-8642-eb1060dd6e1a,16648684,2021-10-26T00:00:00.000+0200
...,...,...,...,...
60,DSQe031,25decfa4-d6e0-d757-722e-0f9db423fc2d,16651361,2021-10-26T00:00:00.000+0200
81,DSQe057,3dec18f4-e83c-10a4-58d1-a41c58306e19,16649411,2021-10-26T00:00:00.000+0200
80,DSQe061,1a9a903b-9524-f8c2-a943-81a108880e46,16649256,2021-10-26T00:00:00.000+0200
61,DSQe200,ddfec0ee-dcc2-db2c-0de2-a22dfe78c044,16652352,2021-10-26T00:00:00.000+0200


In [133]:
IDs.to_csv('Concert_participants_installations.csv', index=False)

# Now separate live streams from live audience devices

might do this by hand. Alphabetisation is helpful that way. 

In [135]:
confirmed_concert = IDs['Installations'].values
confirmed_concert

array(['98ae5c8a-1fed-7d15-a922-9cc2bd9da9e0',
       'c32d3e7d-8ee6-0a11-b056-cb501edbfc7e',
       '005093c6-7880-8728-1256-26822f05cfb0',
       'f045e08d-53b9-5e26-ad38-107564d919bc',
       '77d1745b-a57f-d560-8642-eb1060dd6e1a',
       'aa29f63b-7ab9-5376-c677-d7051cdc684c',
       '03fcbb15-14c6-b806-80b8-e2c161654dd6',
       '69ae0c07-45dc-1a79-b81a-ddb8a020be3c',
       '9197305e-476d-1858-10a4-9658a5333b19',
       '01393de0-e72e-5e80-f06f-8fdb5d0080cb',
       '9ecdcced-debc-78df-3ace-7bc5ebbcb084',
       '68cd78ff-286c-e9a3-dcfa-af2063147700',
       '8672e7d3-3fcc-d38c-3e3c-4261bc59e44f',
       'f03ce18b-d3d9-2639-d0b6-ebca90128a68',
       '278e797b-f81d-27bc-0ab5-7763b027915a',
       '49472968-4242-c1b7-98c2-3a5c60c33d39',
       '981035ba-8cf0-6c34-97ca-fbd61266e8bf',
       'cc1ce8ab-eee9-f8fe-177e-6c9721798373',
       '01d00c32-d4dc-8ce6-353c-43143401adf3',
       '47bf00fa-1f9b-33e1-038b-0e0f19e96d93',
       'abf053b1-19ab-912a-60a2-52d43c1fb79f',
       '2e904

In [138]:
os.listdir()

['PreConcert_MusicLab',
 'PullingMusicLab.ipynb',
 '.DS_Store',
 'Concert_participants_installations.csv',
 'PullingNettskjema.ipynb',
 'CompressedData',
 'README.md',
 '.gitignore',
 'copen_netts.tsv',
 'Test_API',
 'Concert_MusicLab',
 '.ipynb_checkpoints',
 'Copendata',
 '.git',
 'InstOrdData',
 'nettskjema_token.txt']

In [144]:
#interval_folder = 'PreConcert_MusicLab'
interval_folder = 'Concert_MusicLab/'
inst_list = os.listdir(interval_folder)
os.chdir(interval_folder)

In [145]:
os.mkdir('Hall')
os.mkdir('Remote')

In [146]:
for fold in inst_list:
    if fold in confirmed_concert:
        sourcefold = fold
        targetfold = './' + 'Hall' + '/' + fold
        os.rename(sourcefold,targetfold)
    else:
        if len(fold)>10:
            sourcefold = fold
            targetfold = './' + 'Remote' + '/' + fold
            os.rename(sourcefold,targetfold)

In [143]:
os.chdir('..')